In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15695358
paper_name = 'huang_oshea_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load map

In [46]:
original_data = pd.read_excel('raw_data/MATa Collection.xlsx', sheet_name='MATa Collection.xls', skiprows=3)

In [47]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4879 x 10


In [48]:
original_data.head()

,record no.,ORF name,strain,batch,plate,row,column,RG notes,QC Notes,Unnamed: 9
0,338.0,YAL068C,BY4741,01_1,1.0,A,2.0,NaN,NaN,NaN
1,339.0,YAL067C,BY4741,01_1,1.0,A,3.0,NaN,NaN,NaN
2,340.0,YAL066W,BY4741,01_1,1.0,A,4.0,NaN,NaN,NaN
3,341.0,YAL065C,BY4741,01_1,1.0,A,5.0,NaN,NaN,NaN
4,345.0,YAL062W,BY4741,01_1,1.0,A,6.0,NaN,NaN,NaN


In [49]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [50]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [51]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [52]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [53]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             record no. ORF name strain batch  plate  row  column RG notes  \
index_input                                                                  
4705                NaN      NaN    NaN   NaN    NaN  NaN     NaN      NaN   

            QC Notes Unnamed: 9  orf  
index_input                           
4705             NaN        NaN  NAN  


In [54]:
original_data = original_data.loc[t,:]

In [56]:
original_data = original_data.loc[original_data['column'].notnull() & original_data['row'].notnull() & original_data['plate'].notnull()]

In [58]:
original_data['plate'] = original_data['plate'].astype(int)
original_data['column'] = original_data['column'].astype(int)

In [61]:
original_data.set_index(['plate','row','column'], inplace=True)

In [63]:
original_data = original_data[['orf']].copy()

In [64]:
original_data.head()

orf
plate row column         
1     A   2       YAL068C
          3       YAL067C
          4       YAL066W
          5       YAL065C
          6       YAL062W

In [ ]:
original_data = original_data.groupby(original_data.index).mean()

In [ ]:
original_data.shape

# Load data

In [65]:
dt = pd.read_excel('raw_data/Initial Screen Data.xls', sheet_name='Sheet1', skiprows=2)

In [66]:
dt.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96
0,Plate #01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
2,0',0,0,-4.4843,-4.16667,-4.54545,-4.40529,-8.96861,-4.06504,0,...,-4.60829,-4.08163,0,-8.33333,-9.09091,-4.01606,-3.7037,-10.2041,-10.2564,-4.44444
3,120',0,17.2414,-9.25926,0,9.52381,0,-9.17431,0,69.3069,...,29.1971,0,17.2414,-14.9254,-8.54701,8.84956,-9.61538,-9.52381,-8.92857,-8.92857
4,240',0,116.071,80.6452,94.0594,101.01,75.4717,105.263,0,310.976,...,133.333,78.8382,110,77.9221,122.066,70.4225,90.9091,90.4255,88.67,64.1711


In [67]:
dt['Unnamed: 0'] = dt['Unnamed: 0'].astype(str)

In [68]:
ix_plates = dt.loc[dt['Unnamed: 0'].str.startswith('Plate')].index.values

In [81]:
dt_this_list = []
for ix in ix_plates:
    plate_num = int(dt.loc[ix,'Unnamed: 0'].split('#')[1])
    dt_this = dt.loc[ix+1:ix+5,:].T
    
    dt_this.columns = ['pos','0','120','240','360']
    dt_this.drop(index='Unnamed: 0', inplace=True)
    
    dt_this['plate'] = plate_num
    dt_this['row'] = dt_this['pos'].apply(lambda x: x[0])
    dt_this['column'] = dt_this['pos'].apply(lambda x: int(x[1:]))

    dt_this.set_index(['plate','row','column'], inplace=True)
    dt_this.drop(columns=['pos'], inplace=True)
    
    dt_this_list.append(dt_this)

In [82]:
dt = pd.concat(dt_this_list, axis=0)

In [83]:
dt.shape

(5088, 4)

In [87]:
original_data = dt.join(original_data, how='left')

In [89]:
original_data = original_data.loc[original_data['orf'].notnull()]

In [90]:
original_data.shape

(4847, 5)

In [92]:
original_data.head()

0      120      240      360      orf
plate row column                                             
1     A   2             0  17.2414  116.071  207.358  YAL068C
          3       -4.4843 -9.25926  80.6452  138.776  YAL067C
          4      -4.16667        0  94.0594  164.875  YAL066W
          5      -4.54545  9.52381   101.01  176.056  YAL065C
          6      -4.40529        0  75.4717  136.213  YAL062W

In [93]:
original_data.set_index('orf', inplace=True)

In [95]:
original_data['data'] = original_data[['0','120','240','360']].mean(axis=1)

In [96]:
original_data = original_data[['data']].copy()

In [98]:
original_data = original_data.groupby(original_data.index).mean()

In [99]:
original_data.head()

,data
orf,
YAL002W,62.844439
YAL004W,281.418535
YAL005C,343.898165
YAL007C,53.710962
YAL008W,41.967560


In [100]:
original_data.shape

(4760, 1)

# Prepare the final dataset

In [101]:
data = original_data.copy()

In [102]:
dataset_ids = [139]
datasets = datasets.reindex(index=dataset_ids)

In [103]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [104]:
data.head()

dataset_id,139
data_type,value
orf,
YAL002W,62.844439
YAL004W,281.418535
YAL005C,343.898165
YAL007C,53.710962
YAL008W,41.967560


## Subset to the genes currently in SGD

In [105]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [106]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,139
,data_type,value
gene_id,orf,
2,YAL002W,62.844439
1863,YAL004W,281.418535
4,YAL005C,343.898165
5,YAL007C,53.710962
6,YAL008W,41.967560


# Normalize

In [107]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [108]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [109]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id              139          
data_type             value    valuez
gene_id orf                          
2       YAL002W   62.844439 -0.481289
1863    YAL004W  281.418535  2.788341
4       YAL005C  343.898165  3.722968
5       YAL007C   53.710962 -0.617916
6       YAL008W   41.967560 -0.793585

# Print out

In [110]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [111]:
from IO.save_data_to_db3 import *

In [112]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15695358...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.19s/it]

Updating the data_modified_on field...
